# Data Wrangling Exercise (Solution)

Adapted from Dipanjan Sarkar et al. 2018. [Practical Machine Learning with Python](https://link.springer.com/book/10.1007/978-1-4842-3207-1).

## Overview

Data wrangling or data munging is the process of cleaning, transforming, and mapping data from one form to another to utilize it for tasks such as analytics, summarization, reporting, visualization, and so on.

Data wrangling is one of most important and involving steps in the whole Data Science workflow. The output of this process directly impacts all downstream steps such as exploration, summarization, visualization, analysis and even the final result. This clearly shows why Data Scientists spend a lot of time in Data Collection and Wrangling.

## Learning Objectives

- Learn to clean and transform data using pandas
- Master key data wrangling operations:
  - Renaming and rearranging columns
  - Filtering data
  - Handling missing values
  - Managing duplicates
  - Encoding categorical variables
  - Normalizing numeric values
- Perform data summarization and aggregation

### Tasks to be completed

- Clean column names
- Sort and filter data
- Handle missing values
- Remove duplicates
- Encode categorical variables
- Normalize numeric features
- Perform data aggregation

## Prerequisites

- Python programming environment
- Basic understanding of statistical and machine learning concepts
- Familiarity with common ML libraries


## Get Started
- Please select kernel "conda_python3" from SageMaker notebook instance.

### Import necessary libraries


In [80]:
# Import required libraries
import numpy as np  # NumPy for numerical operations
import pandas as pd  # Pandas for handling tabular data
from IPython.display import display  # Allows displaying objects in Jupyter Notebook outputs
from sklearn import preprocessing  # Import preprocessing utilities from scikit-learn

# Suppress chained assignment warning in Pandas
# This warning occurs when modifying a slice of a DataFrame, which can sometimes lead to unexpected behavior.
# Setting it to None disables the warning but requires careful handling to avoid unintended data modifications.
pd.options.mode.chained_assignment = None  

## Data wrangling utility functions


In [81]:
def describe_dataframe(df=pd.DataFrame()):
    """
    This function generates descriptive statistics for a given dataframe.
    
    Args:
        df (pd.DataFrame): The dataframe to be analyzed. Defaults to an empty DataFrame.
    
    Returns:
        None
    """

    # Print section header
    print("\n\n")
    print("*" * 30)
    print("About the Data")
    print("*" * 30)

    # Print the number of rows and columns in the dataframe
    print("Number of rows::", df.shape[0])
    print("Number of columns::", df.shape[1])
    print("\n")

    # Print column names
    print("Column Names::", df.columns.values.tolist())
    print("\n")

    # Print data types of each column
    print



def cleanup_column_names(df, rename_dict={}, do_inplace=True):
    """This function renames columns of a pandas dataframe
       It converts column names to snake case if rename_dict is not passed.
    Args:
        rename_dict (dict): keys represent old column names and values point to
                            newer ones
        do_inplace (bool): flag to update existing dataframe or return a new one
    Returns:
        pandas dataframe if do_inplace is set to False, None otherwise

    """
    # If rename_dict is empty or None, apply automatic column renaming
    if not rename_dict:  
        return df.rename(
            columns={
                # Convert column names to lowercase and replace spaces and slashes with underscores
                col: col.lower().replace(" ", "_").replace(r"/", "_")  
                for col in df.columns.values.tolist()
            },
            inplace=do_inplace,  # Apply renaming in place if do_inplace is True
        )
    else:
        # If rename_dict is provided, use it directly for renaming
        return df.rename(columns=rename_dict, inplace=do_inplace)

## Wine recognition dataset

This is UCI ML Wine recognition datasets. https://archive.ics.uci.edu/ml/machine-learning-databases/wine/wine.data

The data is the results of a chemical analysis of wines grown in the same region in Italy by three different cultivators. There are thirteen different measurements taken for different constituents found in the three types of wine.

Original Owners:

Forina, M. et al, PARVUS - An Extendible Package for Data Exploration, Classification and Correlation. Institute of Pharmaceutical and Food Analysis and Technologies, Via Brigata Salerno, 16147 Genoa, Italy.

Citation:

Lichman, M. (2013). UCI Machine Learning Repository [https://archive.ics.uci.edu/ml]. Irvine, CA: University of California, School of Information and Computer Science.

Dataset characteristics:

- Number of Instances: 178 (50 in each of three classes)
- Number of Attributes: 13 numeric, predictive attributes and the class
- Attribute info:

1. **Alcohol**: alcohol content, reported in units of ABV (alcohol by volume).

1. **Malic acid**: one of the principal organic acids found in wine. Although found in nearly every fruit and berry, it’s flavor is most prominent in green apples; likewise, it projects this sour flavor into wine. For more information, feel free to read about acids in wine.

1. **Ash**: yep, wine has ash in it. Ash is simply the inorganic matter left after evaporation and incineration.

1. **Alcalinity of ash**: the alkalinity of ash determines how basic (as opposed to acidic) the ash in a wine is.

1. **Magnesium**: magnesium is a metal that affects the flavor of wine.

1. **Total phenols**: Phenols are chemicals that affect the taste, color, and mouthfeel (i.e., texture) of wine. For some (very) in-depth information about phenols, we refer you to phenolic content in wine.

1. **Flavoids**: flavonoids are a type of phenol.

1. **Nonflavoid phenols**: nonflavonoids are another type of phenol.

1. **Proanthocyanins**: proanthocyanidins are yet another type of phenol.

1. **Color intensity**: the color intensity of a wine: i.e., how dark it is.

1. **Hue**: the hue of a wine, which is typically determined by the color of the cultivar used (although this is not always the case).

1. **OD280/OD315 of diluted wines**: protein content measurements.

1. **Proline**: an amino acid present in wines.

- Class
  - Class 0: 59
  - Class 1: 71
  - Class 2: 48

'messy_wine_data.csv' is a modified from 'Wine recognition dataset' by introducing some missing values.


## Import data

In [82]:
# Define the file path for the messy wine dataset
messy_wine_data = "../../Data/messy_wine_data.csv"

# Load the dataset into a Pandas DataFrame
df = pd.read_csv(messy_wine_data)

# Display the first few rows of the dataset to inspect its contents
df.head()

,Alcohol,Malic acid,Ash,Alcalinity of ash,Magnesium,Total phenols,Flavanoids,Nonflavanoid phenols,Proanthocyanins,Color intensity,Hue,OD280/OD315 of diluted wines,Proline,Class
0,14.23,NaN,2.43,15.6,127.0,2.80,NaN,0.28,2.29,5.64,1.04,3.92,1065.0,0
1,13.20,1.78,2.14,11.2,100.0,2.65,NaN,0.26,1.28,4.38,1.05,3.40,1050.0,0
2,13.16,2.36,2.67,18.6,NaN,2.80,3.24,0.30,2.81,5.68,1.03,3.17,1185.0,0
3,14.37,1.95,2.50,16.8,113.0,3.85,3.49,0.24,2.18,7.80,0.86,3.45,1480.0,0
4,13.24,2.59,2.87,21.0,NaN,2.80,2.69,0.39,-1.00,4.32,1.04,2.93,735.0,0


In [83]:
# Call the describe_dataframe function to generate statistical summaries  
# of the given DataFrame (df). This typically includes metrics like mean, 
# standard deviation, min, max, and quartiles for numerical columns.
describe_dataframe(df)




******************************
About the Data
******************************
Number of rows:: 178
Number of columns:: 14


Column Names:: ['Alcohol', 'Malic acid', 'Ash', 'Alcalinity of ash', 'Magnesium', 'Total phenols', 'Flavanoids', 'Nonflavanoid phenols', 'Proanthocyanins', 'Color intensity', 'Hue', 'OD280/OD315 of diluted wines', 'Proline', 'Class']




In [84]:
# Print the shape (dimensions) of the DataFrame `df`
# `df.shape` returns a tuple (number of rows, number of columns)
print("Shape of df={}".format(df.shape))

Shape of df=(178, 14)


## Rename Columns


In [85]:
# Print the column names of the DataFrame 'df'.
print("Dataframe columns:\n{}".format(df.columns.tolist()))
# - `df.columns` accesses the column index of the DataFrame 'df'.
# - `.tolist()` converts the column index (which is a Pandas Index object) into a Python list.
# - `"{}\n".format(...)` is a string formatting method to insert the list of column names into the string and add a newline character for better readability.
# - `print(...)` function displays the formatted string to the console, showing the list of column names.

Dataframe columns:
['Alcohol', 'Malic acid', 'Ash', 'Alcalinity of ash', 'Magnesium', 'Total phenols', 'Flavanoids', 'Nonflavanoid phenols', 'Proanthocyanins', 'Color intensity', 'Hue', 'OD280/OD315 of diluted wines', 'Proline', 'Class']


In [86]:
# Call cleanup_column_names function on a Pandas DataFrame.
cleanup_column_names(df)

In [87]:
# Print the list of column names of the DataFrame 'df'.
print("Dataframe columns:\n{}".format(df.columns.tolist()))

Dataframe columns:
['alcohol', 'malic_acid', 'ash', 'alcalinity_of_ash', 'magnesium', 'total_phenols', 'flavanoids', 'nonflavanoid_phenols', 'proanthocyanins', 'color_intensity', 'hue', 'od280_od315_of_diluted_wines', 'proline', 'class']


## Sort Rows on defined attributes


In [88]:
# Display the first 5 rows of the DataFrame 'df'.
# This function is useful for quickly inspecting the structure and sample data of the DataFrame.
df.head()

,alcohol,malic_acid,ash,alcalinity_of_ash,magnesium,total_phenols,flavanoids,nonflavanoid_phenols,proanthocyanins,color_intensity,hue,od280_od315_of_diluted_wines,proline,class
0,14.23,NaN,2.43,15.6,127.0,2.80,NaN,0.28,2.29,5.64,1.04,3.92,1065.0,0
1,13.20,1.78,2.14,11.2,100.0,2.65,NaN,0.26,1.28,4.38,1.05,3.40,1050.0,0
2,13.16,2.36,2.67,18.6,NaN,2.80,3.24,0.30,2.81,5.68,1.03,3.17,1185.0,0
3,14.37,1.95,2.50,16.8,113.0,3.85,3.49,0.24,2.18,7.80,0.86,3.45,1480.0,0
4,13.24,2.59,2.87,21.0,NaN,2.80,2.69,0.39,-1.00,4.32,1.04,2.93,735.0,0


In [89]:
# Sort the DataFrame `df` based on two columns:  
# 1. `malic_acid` in ascending order (smallest to largest)  
# 2. `ash` in descending order (largest to smallest)  
sorted_df = df.sort_values(["malic_acid", "ash"], ascending=[True, False])

# Display the first few rows of the sorted DataFrame  
display(sorted_df.head())

,alcohol,malic_acid,ash,alcalinity_of_ash,magnesium,total_phenols,flavanoids,nonflavanoid_phenols,proanthocyanins,color_intensity,hue,od280_od315_of_diluted_wines,proline,class
113,11.41,0.74,2.50,21.0,88.0,2.48,2.01,0.42,1.44,3.08,1.10,2.31,434.0,1
84,11.84,0.89,2.58,18.0,94.0,2.20,2.21,0.22,2.35,3.05,0.79,3.08,520.0,1
76,-10.00,0.90,1.71,16.0,86.0,1.95,NaN,0.24,1.46,4.60,1.19,2.48,392.0,1
80,-10.00,0.92,2.00,19.0,86.0,2.42,2.26,0.30,1.43,2.50,1.38,3.12,1.0,1
68,13.34,0.94,2.36,17.0,110.0,2.53,1.30,0.55,0.42,3.17,1.02,1.93,750.0,1


In [90]:
# Sort the DataFrame by the "alcohol" column in descending order
# The highest alcohol values will appear first
sorted_df = df.sort_values(["alcohol"], ascending=[False])

# Display the first few rows of the sorted DataFrame
display(sorted_df.head())

,alcohol,malic_acid,ash,alcalinity_of_ash,magnesium,total_phenols,flavanoids,nonflavanoid_phenols,proanthocyanins,color_intensity,hue,od280_od315_of_diluted_wines,proline,class
8,14.83,NaN,2.17,14.0,97.0,2.80,NaN,0.29,1.98,5.20,1.08,2.85,1045.0,0
13,14.75,1.73,2.39,11.4,NaN,3.10,3.69,0.43,2.81,5.40,1.25,2.73,1150.0,0
6,14.39,1.87,2.45,14.6,96.0,2.50,2.52,0.30,1.98,5.25,1.02,3.58,1290.0,0
46,14.38,3.59,2.28,16.0,102.0,3.25,3.17,0.27,2.19,4.90,1.04,3.44,1065.0,0
14,14.38,1.87,0.00,12.0,102.0,3.30,3.64,0.29,2.96,7.50,1.20,3.00,1547.0,0


## Rearrange Columns in a Dataframe


In [91]:
# Display the first 5 rows of the DataFrame 'df'.
# This function is useful for quickly inspecting the structure and sample data of the DataFrame.
df.head()

,alcohol,malic_acid,ash,alcalinity_of_ash,magnesium,total_phenols,flavanoids,nonflavanoid_phenols,proanthocyanins,color_intensity,hue,od280_od315_of_diluted_wines,proline,class
0,14.23,NaN,2.43,15.6,127.0,2.80,NaN,0.28,2.29,5.64,1.04,3.92,1065.0,0
1,13.20,1.78,2.14,11.2,100.0,2.65,NaN,0.26,1.28,4.38,1.05,3.40,1050.0,0
2,13.16,2.36,2.67,18.6,NaN,2.80,3.24,0.30,2.81,5.68,1.03,3.17,1185.0,0
3,14.37,1.95,2.50,16.8,113.0,3.85,3.49,0.24,2.18,7.80,0.86,3.45,1480.0,0
4,13.24,2.59,2.87,21.0,NaN,2.80,2.69,0.39,-1.00,4.32,1.04,2.93,735.0,0


In [92]:
# Rearrange the columns in a specific order for better readability and analysis.
# The columns are ordered as: 'class', followed by various chemical properties of the wine.

display(
    df[
        [   # List of columns arranged in the desired order
            "class",  # Target variable representing the wine class/category
            "alcohol",  # Alcohol content in the wine
            "malic_acid",  # Malic acid concentration
            "ash",  # Ash content in the wine
            "alcalinity_of_ash",  # Alkalinity of ash
            "magnesium",  # Magnesium content
            "total_phenols",  # Total phenolic compounds
            "flavanoids",  # Flavonoid content
            "nonflavanoid_phenols",  # Non-flavonoid phenols
            "proanthocyanins",  # Proanthocyanins (a type of phenolic compound)
            "color_intensity",  # Intensity of the wine color
            "hue",  # Hue of the wine
            "od280_od315_of_diluted_wines",  # OD280/OD315 ratio (indicator of wine quality)
            "proline",  # Proline content (an amino acid relevant to wine properties)
        ]
    ].head()  # Display the first few rows of the reordered DataFrame
)

,class,alcohol,malic_acid,ash,alcalinity_of_ash,magnesium,total_phenols,flavanoids,nonflavanoid_phenols,proanthocyanins,color_intensity,hue,od280_od315_of_diluted_wines,proline
0,0,14.23,NaN,2.43,15.6,127.0,2.80,NaN,0.28,2.29,5.64,1.04,3.92,1065.0
1,0,13.20,1.78,2.14,11.2,100.0,2.65,NaN,0.26,1.28,4.38,1.05,3.40,1050.0
2,0,13.16,2.36,2.67,18.6,NaN,2.80,3.24,0.30,2.81,5.68,1.03,3.17,1185.0
3,0,14.37,1.95,2.50,16.8,113.0,3.85,3.49,0.24,2.18,7.80,0.86,3.45,1480.0
4,0,13.24,2.59,2.87,21.0,NaN,2.80,2.69,0.39,-1.00,4.32,1.04,2.93,735.0


In [93]:
# Display the first few rows of the dataframe (head) with columns rearranged in a specified order

# Rearrange the columns of the dataframe in the given order:
# - 'alcohol' (first column)
# - 'color_intensity', 'hue', 'malic_acid', 'ash', 'alcalinity_of_ash', 'magnesium', etc.
# - 'class' is placed last, indicating it is the target variable or label.

# The `display()` function is used to show the top rows of the dataframe.
display(
    df[
        [
            "alcohol",  # Alcohol content of the wine
            "color_intensity",  # Intensity of the wine's color
            "hue",  # Hue (color tone) of the wine
            "malic_acid",  # Amount of malic acid in the wine
            "ash",  # Ash content in the wine
            "alcalinity_of_ash",  # Alcalinity (pH-related) of the ash in the wine
            "magnesium",  # Magnesium content in the wine
            "total_phenols",  # Total phenols (a type of antioxidant) in the wine
            "flavanoids",  # Flavonoids (a type of phenolic compound) in the wine
            "nonflavanoid_phenols",  # Non-flavonoid phenolic compounds
            "proanthocyanins",  # Proanthocyanins (another type of phenolic compound)
            "od280_od315_of_diluted_wines",  # OD280/OD315 (optical density) ratio for the wine
            "proline",  # Proline content, an amino acid that helps in wine taste
            "class",  # The target variable: wine classification (likely wine type)
        ]
    ].head()  # Get the first 5 rows of the reordered dataframe
)

,alcohol,color_intensity,hue,malic_acid,ash,alcalinity_of_ash,magnesium,total_phenols,flavanoids,nonflavanoid_phenols,proanthocyanins,od280_od315_of_diluted_wines,proline,class
0,14.23,5.64,1.04,NaN,2.43,15.6,127.0,2.80,NaN,0.28,2.29,3.92,1065.0,0
1,13.20,4.38,1.05,1.78,2.14,11.2,100.0,2.65,NaN,0.26,1.28,3.40,1050.0,0
2,13.16,5.68,1.03,2.36,2.67,18.6,NaN,2.80,3.24,0.30,2.81,3.17,1185.0,0
3,14.37,7.80,0.86,1.95,2.50,16.8,113.0,3.85,3.49,0.24,2.18,3.45,1480.0,0
4,13.24,4.32,1.04,2.59,2.87,21.0,NaN,2.80,2.69,0.39,-1.00,2.93,735.0,0


## Filtering Columns

Using Column Index


In [94]:
# Access the DataFrame's 3rd column (index 3) using iloc
# iloc[:, 3] selects all rows from the column at index 3
# .values returns the underlying NumPy array for that column
# [0:10] slices the first 10 values from the array
print(df.iloc[:, 3].values[0:10])  # Print the first 10 values from the 3rd column

[15.6 11.2 18.6 16.8 21.  15.2 14.6 17.6 14.  16. ]


### Using Column Name


In [95]:
# Print the first 10 values of the 'total_phenols' column from the DataFrame
# 'df' is assumed to be the DataFrame containing the data
print(df.total_phenols.values[0:10])  # Accesses 'total_phenols' column and prints the first 10 values

[2.8  2.65 2.8  3.85 2.8  3.27 2.5  2.6  2.8  2.98]


### Using Column Datatype


In [96]:
# Select columns with data type 'float64' from the DataFrame (df)
# `select_dtypes(include=["float64"])` filters columns that are of type float64
# This will return a DataFrame containing only the float64 columns
float_columns = df.select_dtypes(include=["float64"])

# Print the first 10 values of the first float64 column (from the filtered DataFrame)
# `values` gives the underlying NumPy array of the DataFrame
# `[:10, 0]` selects the first 10 rows and the first column (0-indexed) from the NumPy array
print(float_columns.values[:10, 0])

[ 14.23  13.2   13.16  14.37  13.24  14.2   14.39  14.06  14.83 -10.  ]


## Filtering Rows

Select specific rows


In [97]:
# Select specific rows by their index positions: 21, 45, and 100
# The .iloc[] method is used to access rows by their integer index positions (0-based index)
display(df.iloc[[21, 45, 100]])  # Display rows 21, 45, and 100 from the DataFrame 'df'

,alcohol,malic_acid,ash,alcalinity_of_ash,magnesium,total_phenols,flavanoids,nonflavanoid_phenols,proanthocyanins,color_intensity,hue,od280_od315_of_diluted_wines,proline,class
21,12.93,3.80,2.65,18.6,102.0,2.41,2.41,0.25,1.98,4.50,1.03,3.52,770.0,0
45,14.21,4.04,2.44,18.9,111.0,2.85,2.65,0.30,1.25,5.24,0.87,3.33,1080.0,0
100,12.08,2.08,1.70,17.5,97.0,2.23,2.17,0.26,1.40,3.30,1.27,2.96,710.0,1


### Exclude Specific Row indices


In [98]:
# drop the first and third rows (index 0 and index 2)
# The 'axis=0' specifies that we are working with rows (not columns).
# 'drop' method removes the rows from the DataFrame based on their index values.
display(df.drop([0, 2], axis=0).head())  # Display the first few rows of the modified DataFrame

,alcohol,malic_acid,ash,alcalinity_of_ash,magnesium,total_phenols,flavanoids,nonflavanoid_phenols,proanthocyanins,color_intensity,hue,od280_od315_of_diluted_wines,proline,class
1,13.20,1.78,2.14,11.2,100.0,2.65,NaN,0.26,1.28,4.38,1.05,3.40,1050.0,0
3,14.37,1.95,2.50,16.8,113.0,3.85,3.49,0.24,2.18,7.80,0.86,3.45,1480.0,0
4,13.24,2.59,2.87,21.0,NaN,2.80,2.69,0.39,-1.00,4.32,1.04,2.93,735.0,0
5,14.20,1.76,2.45,15.2,112.0,3.27,3.39,0.34,1.97,6.75,1.05,2.85,1450.0,0
6,14.39,1.87,2.45,14.6,96.0,2.50,2.52,0.30,1.98,5.25,1.02,3.58,1290.0,0


### Conditional Filtering


In [99]:
# Filter the rows where the 'ash' column value is greater than 2
# Then display the first 5 rows of the resulting DataFrame
display(df[df.ash > 2].head())

,alcohol,malic_acid,ash,alcalinity_of_ash,magnesium,total_phenols,flavanoids,nonflavanoid_phenols,proanthocyanins,color_intensity,hue,od280_od315_of_diluted_wines,proline,class
0,14.23,NaN,2.43,15.6,127.0,2.80,NaN,0.28,2.29,5.64,1.04,3.92,1065.0,0
1,13.20,1.78,2.14,11.2,100.0,2.65,NaN,0.26,1.28,4.38,1.05,3.40,1050.0,0
2,13.16,2.36,2.67,18.6,NaN,2.80,3.24,0.30,2.81,5.68,1.03,3.17,1185.0,0
3,14.37,1.95,2.50,16.8,113.0,3.85,3.49,0.24,2.18,7.80,0.86,3.45,1480.0,0
4,13.24,2.59,2.87,21.0,NaN,2.80,2.69,0.39,-1.00,4.32,1.04,2.93,735.0,0


### Offset from top of the dataframe


In [100]:
# Skip the top 100 rows of the DataFrame `df` and display the first 5 rows of the remaining data
display(df[100:].head())  # `df[100:]` slices the DataFrame to start from the 101st row, and `head()` shows the first 5 rows

,alcohol,malic_acid,ash,alcalinity_of_ash,magnesium,total_phenols,flavanoids,nonflavanoid_phenols,proanthocyanins,color_intensity,hue,od280_od315_of_diluted_wines,proline,class
100,12.08,2.08,1.70,17.5,97.0,2.23,2.17,0.26,1.40,3.30,1.27,2.96,710.0,1
101,12.60,1.34,1.90,18.5,88.0,1.45,1.36,0.29,1.35,2.45,1.04,2.77,562.0,1
102,12.34,2.45,2.46,21.0,98.0,2.56,2.11,0.34,1.31,2.80,0.80,3.38,438.0,1
103,11.82,1.72,1.88,19.5,86.0,2.50,1.64,0.37,1.42,2.06,0.94,2.44,415.0,1
104,12.51,1.73,1.98,20.5,85.0,2.20,1.92,0.32,1.48,2.94,1.04,3.57,672.0,1


### Offset from bottom of the dataframe


In [101]:
# Skip the last 10 rows of the DataFrame 'df' and display the first few rows of the remaining data
# The slicing `df[:-10]` skips the last 10 rows by specifying a range that goes up to the 10th-to-last row
display(df[:-10].head())  # Display the first few rows of the modified DataFrame

,alcohol,malic_acid,ash,alcalinity_of_ash,magnesium,total_phenols,flavanoids,nonflavanoid_phenols,proanthocyanins,color_intensity,hue,od280_od315_of_diluted_wines,proline,class
0,14.23,NaN,2.43,15.6,127.0,2.80,NaN,0.28,2.29,5.64,1.04,3.92,1065.0,0
1,13.20,1.78,2.14,11.2,100.0,2.65,NaN,0.26,1.28,4.38,1.05,3.40,1050.0,0
2,13.16,2.36,2.67,18.6,NaN,2.80,3.24,0.30,2.81,5.68,1.03,3.17,1185.0,0
3,14.37,1.95,2.50,16.8,113.0,3.85,3.49,0.24,2.18,7.80,0.86,3.45,1480.0,0
4,13.24,2.59,2.87,21.0,NaN,2.80,2.69,0.39,-1.00,4.32,1.04,2.93,735.0,0


## TypeCasting/Data Type Conversion


In [102]:
# Print the data types of the columns in the original dataframe
print("Old dtypes:\n", df.dtypes)

# Change the data type of the 'hue' column from object (string) to integer (int)
df["hue"] = df["hue"].astype(int)

# Print the data types of the columns in the dataframe after the change
print("New dtypes:\n", df.dtypes)

Old dtypes:
 alcohol                         float64
malic_acid                      float64
ash                             float64
alcalinity_of_ash               float64
magnesium                       float64
total_phenols                   float64
flavanoids                      float64
nonflavanoid_phenols            float64
proanthocyanins                 float64
color_intensity                 float64
hue                             float64
od280_od315_of_diluted_wines    float64
proline                         float64
class                             int64
dtype: object
New dtypes:
 alcohol                         float64
malic_acid                      float64
ash                             float64
alcalinity_of_ash               float64
magnesium                       float64
total_phenols                   float64
flavanoids                      float64
nonflavanoid_phenols            float64
proanthocyanins                 float64
color_intensity                 float64


## Missing Values

_Note: you make get some `FutureWarning` notifications in the following cells. They shouldn't cause problems._


In [103]:
# Drop rows with missing values in the 'malic_acid' column
df_dropped = df.dropna(subset=["malic_acid"])  # This will remove any rows where 'malic_acid' is NaN

# Display the shape of the DataFrame after dropping rows
df_dropped.shape  # This will return the number of rows and columns in the new DataFrame

(165, 14)

In [104]:
# Fill missing 'magnesium' values in the df_dropped DataFrame with the mean of 'magnesium' column
# We use np.round to round the mean to 2 decimal places

# Calculate the mean of the "magnesium" column and round it to 2 decimal places
mean_magnesium = np.round(df["magnesium"].mean(), decimals=2)

# Fill missing values in the "magnesium" column with the mean
df_dropped["magnesium"].fillna(value=mean_magnesium, inplace=True)

# Identify rows where values were replaced
replaced_rows = df["magnesium"].isna() & df_dropped["magnesium"].notna()

# Display the rows and columns where values were replaced
replaced_values = df_dropped.loc[replaced_rows, "magnesium"]
print("Values replaced with the mean:")
print(replaced_values)

Values replaced with the mean:
2      99.26
4      99.26
13     99.26
17     99.26
43     99.26
48     99.26
70     99.26
71     99.26
73     99.26
90     99.26
121    99.26
150    99.26
Name: magnesium, dtype: float64


/var/folders/6k/c9mt0r2j39x2lf40w6rs3lfr0000gq/T/ipykernel_15061/2551384559.py:8: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  df_dropped["magnesium"].fillna(value=mean_magnesium, inplace=True)


In [105]:
# Fill missing values in the "flavanoids" column using the previous row's value (forward fill)
# This method propagates the last valid observation forward to the next missing value
df_dropped["flavanoids"].fillna(method="ffill", inplace=True)

/var/folders/6k/c9mt0r2j39x2lf40w6rs3lfr0000gq/T/ipykernel_15061/3959220245.py:3: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  df_dropped["flavanoids"].fillna(method="ffill", inplace=True)
/var/folders/6k/c9mt0r2j39x2lf40w6rs3lfr0000gq/T/ipykernel_15061/3959220245.py:3: FutureWarning: Series.fillna with 'method' is deprecated and will raise in a future version. Use obj.ffill() or obj.bfill() instead.
  df_dropped["flavanoids"].fillna(method="ffill", inplace=True)


In [106]:
# Fill missing values in the "flavanoids" column by using the value from the next row (backward fill)
df_dropped["flavanoids"].fillna(method="bfill", inplace=True)
# `method="bfill"` specifies to fill the missing values using the value from the next row.
# `inplace=True` modifies the DataFrame directly, without creating a new one.

/var/folders/6k/c9mt0r2j39x2lf40w6rs3lfr0000gq/T/ipykernel_15061/2033178559.py:2: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  df_dropped["flavanoids"].fillna(method="bfill", inplace=True)
/var/folders/6k/c9mt0r2j39x2lf40w6rs3lfr0000gq/T/ipykernel_15061/2033178559.py:2: FutureWarning: Series.fillna with 'method' is deprecated and will raise in a future version. Use obj.ffill() or obj.bfill() instead.
  df_dropped["flavanoids"].fillna(method="bfill", inplace=True)


## Duplicates


In [107]:
# Display the first few rows of the dataframe before dropping duplicates
display(df_dropped.head())

# Print the shape (number of rows and columns) of the dataframe before dropping duplicates
print("Shape of df before dropping duplicates ={}".format(df_dropped.shape))

,alcohol,malic_acid,ash,alcalinity_of_ash,magnesium,total_phenols,flavanoids,nonflavanoid_phenols,proanthocyanins,color_intensity,hue,od280_od315_of_diluted_wines,proline,class
1,13.20,1.78,2.14,11.2,100.00,2.65,3.24,0.26,1.28,4.38,1,3.40,1050.0,0
2,13.16,2.36,2.67,18.6,99.26,2.80,3.24,0.30,2.81,5.68,1,3.17,1185.0,0
3,14.37,1.95,2.50,16.8,113.00,3.85,3.49,0.24,2.18,7.80,0,3.45,1480.0,0
4,13.24,2.59,2.87,21.0,99.26,2.80,2.69,0.39,-1.00,4.32,1,2.93,735.0,0
5,14.20,1.76,2.45,15.2,112.00,3.27,3.39,0.34,1.97,6.75,1,2.85,1450.0,0


Shape of df before dropping duplicates =(165, 14)


In [108]:
# Drop duplicate rows based on the 'alcohol' column
df_dropped.drop_duplicates(subset=["alcohol"], inplace=True)

# Display the updated dataframe (after duplicates are dropped) to verify the changes
display(df_dropped.head())

# Print the shape (number of rows and columns) of the dataframe after dropping duplicates
print("Shape of df after dropping duplicates ={}".format(df_dropped.shape))

,alcohol,malic_acid,ash,alcalinity_of_ash,magnesium,total_phenols,flavanoids,nonflavanoid_phenols,proanthocyanins,color_intensity,hue,od280_od315_of_diluted_wines,proline,class
1,13.20,1.78,2.14,11.2,100.00,2.65,3.24,0.26,1.28,4.38,1,3.40,1050.0,0
2,13.16,2.36,2.67,18.6,99.26,2.80,3.24,0.30,2.81,5.68,1,3.17,1185.0,0
3,14.37,1.95,2.50,16.8,113.00,3.85,3.49,0.24,2.18,7.80,0,3.45,1480.0,0
4,13.24,2.59,2.87,21.0,99.26,2.80,2.69,0.39,-1.00,4.32,1,2.93,735.0,0
5,14.20,1.76,2.45,15.2,112.00,3.27,3.39,0.34,1.97,6.75,1,2.85,1450.0,0


Shape of df after dropping duplicates =(113, 14)


## Encode Categoricals


In [109]:
# Get One Hot Encoding using get_dummies() for 'class'
# The get_dummies() function converts categorical variable(s) into dummy/indicator variables
# Here, we are applying it to the 'class' column of the DataFrame 'df'

# 'columns=["class"]' specifies that the 'class' column will be encoded
display(pd.get_dummies(df, columns=["class"]).head())
# The 'head()' function displays the first 5 rows of the resulting DataFrame after encoding

,alcohol,malic_acid,ash,alcalinity_of_ash,magnesium,total_phenols,flavanoids,nonflavanoid_phenols,proanthocyanins,color_intensity,hue,od280_od315_of_diluted_wines,proline,class_0,class_1,class_2
0,14.23,NaN,2.43,15.6,127.0,2.80,NaN,0.28,2.29,5.64,1,3.92,1065.0,True,False,False
1,13.20,1.78,2.14,11.2,100.0,2.65,NaN,0.26,1.28,4.38,1,3.40,1050.0,True,False,False
2,13.16,2.36,2.67,18.6,NaN,2.80,3.24,0.30,2.81,5.68,1,3.17,1185.0,True,False,False
3,14.37,1.95,2.50,16.8,113.0,3.85,3.49,0.24,2.18,7.80,0,3.45,1480.0,True,False,False
4,13.24,2.59,2.87,21.0,NaN,2.80,2.69,0.39,-1.00,4.32,1,2.93,735.0,True,False,False


## Random Sampling data from DataFrame


In [110]:
# Randomly sample 30% of the rows from the DataFrame (with replacement)
# frac=0.3: Specifies the fraction of rows to sample (30%)
# replace=True: Sampling is done with replacement, meaning the same row can be sampled multiple times
# random_state=42: Ensures reproducibility by setting a fixed seed for the random number generator
# display(): Used to show the first few rows of the randomly sampled DataFrame
display(df.sample(frac=0.3, replace=True, random_state=42).head())

,alcohol,malic_acid,ash,alcalinity_of_ash,magnesium,total_phenols,flavanoids,nonflavanoid_phenols,proanthocyanins,color_intensity,hue,od280_od315_of_diluted_wines,proline,class
102,12.34,2.45,2.46,21.0,98.0,2.56,2.11,0.34,1.31,2.80,0,3.38,438.0,1
92,12.69,1.53,2.26,20.7,80.0,1.38,1.46,0.58,1.62,3.05,0,2.06,495.0,1
14,14.38,1.87,0.00,12.0,102.0,3.30,3.64,0.29,2.96,7.50,1,3.00,1547.0,0
106,12.25,1.73,2.12,19.0,80.0,1.65,2.03,0.37,1.63,3.40,1,3.17,510.0,1
71,13.86,1.51,2.67,25.0,NaN,2.95,2.86,0.21,1.87,3.38,1,3.16,410.0,1


## Normalizing Numeric Values

Normalizing values is a crucial step in machine learning to ensure that all features contribute equally to the model's learning process. Features with larger magnitude values can dominate the model, leading to bias and poor performance, especially in algorithms sensitive to scale, such as gradient-based methods (e.g., linear regression, neural networks) and distance-based algorithms (e.g., k-Nearest Neighbors, k-Means). Normalization scales features to a standard range, such as [0, 1] or a mean of 0 and a standard deviation of 1, which helps improve convergence speed, enhances model accuracy, and ensures fair comparison across features. Additionally, normalization reduces the impact of outliers, making the model more robust and reliable. By standardizing the data, normalization enables algorithms to perform optimally, leading to better generalization and interpretability of results.

Normalize 'alcohol' values using **Min-Max Scaler**


In [111]:
# Normalize 'alcohol' values using Min-Max Scaler

# Create a copy of the dataframe without missing values to avoid altering the original data
df_normalized = df.dropna().copy()

# Create an instance of MinMaxScaler from sklearn.preprocessing to scale data between 0 and 1
min_max_scaler = preprocessing.MinMaxScaler()

# Transform the 'alcohol' column values:
# - Reshape the data to (-1, 1) to ensure it’s in the correct 2D format for the scaler
# - Fit the MinMaxScaler to the 'alcohol' values and then transform them into scaled values
alcohol_scaled = min_max_scaler.fit_transform(
    df_normalized["alcohol"].values.reshape(-1, 1)
)

# Replace the original 'alcohol' column with the scaled values
df_normalized["alcohol"] = alcohol_scaled.reshape(-1, 1)

In [112]:
# Display the first few rows of the normalized DataFrame to check the results
display(df_normalized.head())

,alcohol,malic_acid,ash,alcalinity_of_ash,magnesium,total_phenols,flavanoids,nonflavanoid_phenols,proanthocyanins,color_intensity,hue,od280_od315_of_diluted_wines,proline,class
3,0.99918,1.95,2.50,16.8,113.0,3.85,3.49,0.24,2.18,7.80,0,3.45,1480.0,0
5,0.99221,1.76,2.45,15.2,112.0,3.27,3.39,0.34,1.97,6.75,1,2.85,1450.0,0
6,1.00000,1.87,2.45,14.6,96.0,2.50,2.52,0.30,1.98,5.25,1,3.58,1290.0,0
11,0.98893,1.48,0.00,16.8,95.0,2.20,2.43,0.26,1.57,5.00,1,2.82,1280.0,0
12,0.97376,1.73,2.41,16.0,89.0,2.60,2.76,0.29,1.81,5.60,1,2.90,1320.0,0


### Normalize quantity purchased values using **Robust Scaler**


In [113]:
# Normalize 'magnesium' values using Robust Scaler

# Create a copy of the DataFrame with any NaN values dropped for clean data
df_normalized = df.dropna().copy()

# Create a RobustScaler instance (this scales data by removing the median and scaling according to the interquartile range)
robust_scaler = preprocessing.RobustScaler()

# Apply RobustScaler to the 'magnesium' column by reshaping it into a 2D array (required by the scaler)
# The .fit_transform() method fits the scaler and applies it to normalize the 'magnesium' values
magnesium_scaled = robust_scaler.fit_transform(
    df_normalized["magnesium"].values.reshape(-1, 1)
)

# Replace the original 'magnesium' values in the DataFrame with the scaled values
df_normalized["magnesium"] = magnesium_scaled.reshape(-1, 1)

In [114]:
# Display the first few rows (default 5 rows) of the DataFrame to inspect the normalized data
display(df_normalized.head())

,alcohol,malic_acid,ash,alcalinity_of_ash,magnesium,total_phenols,flavanoids,nonflavanoid_phenols,proanthocyanins,color_intensity,hue,od280_od315_of_diluted_wines,proline,class
3,14.37,1.95,2.50,16.8,0.842105,3.85,3.49,0.24,2.18,7.80,0,3.45,1480.0,0
5,14.20,1.76,2.45,15.2,0.789474,3.27,3.39,0.34,1.97,6.75,1,2.85,1450.0,0
6,14.39,1.87,2.45,14.6,-0.052632,2.50,2.52,0.30,1.98,5.25,1,3.58,1290.0,0
11,14.12,1.48,0.00,16.8,-0.105263,2.20,2.43,0.26,1.57,5.00,1,2.82,1280.0,0
12,13.75,1.73,2.41,16.0,-0.421053,2.60,2.76,0.29,1.81,5.60,1,2.90,1320.0,0


## Data Summarization

Condition based aggregation


In [115]:
# Get the mean 'hue' of class 1 wine
# Filter the dataframe to select only rows where the 'class' column is equal to 1,
# then select the 'hue' column from those rows and calculate the mean.
mean_hue = df["hue"][df["class"] == 1].mean()

# Print the mean 'hue' value for class 1 wine
print("Mean 'hue' of class 1 wine :: {}".format(mean_hue))

Mean 'hue' of class 1 wine :: 0.5633802816901409


In [116]:
# Filter the rows where the 'class' column is equal to 0 (class 0 wine)
# Then, select the 'alcohol' column from these filtered rows
# Finally, get the maximum value from the 'alcohol' column for class 0 wine
max_alcohol = df["alcohol"][df["class"] == 0].max()

# Print the result, showing the maximum 'alcohol' value for class 0 wine
print("Max 'alcohol' of class 0 wine :: {}".format(max_alcohol))

Max 'alcohol' of class 0 wine :: 14.83


## Conclusion

Through this exercise, we learned essential data wrangling techniques including:

- Data cleaning and transformation
- Column manipulation
- Row filtering
- Missing value handling
- Data type conversion
- Categorical encoding
- Numeric value normalization
- Data summarization

## Clean up

Remember to shut down your Jupyter Notebook environment and delete any unnecessary files or resources once you've completed the tutorial.
